Universidade Federal de Alagoas

Instituto de Computação

Orientanda: Júlia Albuquerque Aguiar

Orientador: André Lage

[Repositório no Github](https://github.com/juliaguiar/TCC/)

# Código-fonte para protótipo de TCC

## Arquivo `violence.py`

A classe `Violence` tem dois atributos: um `type` que representa o tipo de violência - sexual, física, psicológica, moral ou patrimonial - e um `value` que representa o peso desse tipo; quanto maior o peso, mais provável que esse tipo represente o relato.

In [ ]:
class Violence(object):
  def __init__(self, value, type):
    self.__value = value
    self.__type = type

  def __repr__(self):
    return "value:%s type:%s" % (self.__value, self.__type)

  def get_value(self):
    return self.__value

  def get_type(self):
    return self.__type

## Arquivo `myTweetTokenizer.py`

Nesse arquivo é feita a personalização do tokenizer considerando as especificidades de um texto de tweet, cujo conteúdo tem muitos caracteres distintos do que são tratados por *default* no **SpaCy**: [How spaCy’s tokenizer works](https://spacy.io/usage/linguistic-features#how-tokenizer-works).

In [ ]:
import spacy
from spacy.tokens import Doc
import twikenizer as twk

twk = twk.Twikenizer()

class MyTweetTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, tweet):
        words = twk.tokenize(tweet)
        spaces = [True] * len(words)
        return Doc(self.vocab, words=words, spaces=spaces)

[Repositório](https://github.com/Guilherme-Routar/Twikenizer) do autor da biblioteca e seu [artigo](https://www.voxpol.eu/download/ma_thesis/326963_2.pdf) sobre a pesquisa em que é analisada as diferenças entre o resultado desse trabalho comparado ao tokenizer padrão do **SpaCy** e **NLTK** em *Table 3.3* da seção "3.2 Tweets tokenization".

## Arquivo `classificationViolence.py`

### Imports e Variáveis

Começamos com os imports: `spacy` é a biblioteca de NLP de Python, `tweepy` é a biblioteca para acessar a API do Twitter e `pandas` é uma biblioteca de análise e estrutura de dados com Python que usaremos para criar um *DataFrame*; os outros são para acessar as outras classes do projeto. 

In [ ]:
import spacy
import tweepy
import pandas as pd
from dev.myTweetTokenizer import MyTweetTokenizer
from dev.violence import Violence

Usando `pandas` criamos um _DataFrame_ e colocamos os dados com sua **Data** (`column="data"`) e **Texto** (`column="post"`) que já vêm com a captura das postagens pela API e o **Tipo de Violência** (`column="tipoDeViolencia"`) que é gerado pelo algoritmo: 

In [ ]:
df = pd.DataFrame(columns=["data", "post", "tipoDeViolencia"])

Criamos **dicionários** para cada um dos 5 **tipos de violência**. Esses verbos devem ser representativos sobre qual tipo de violência se trata, ou seja, os verbos mais frequentes nos relatos de cada tipo. Os que estão escritos a seguir foram adicionados apenas para simulação e deverão ser a entrada do sistema para que o usuário final possa adicioná-los:  

In [ ]:
#TODO citar no TCC q vc dividiu nesses tipos de violência de acordo com os tipos categorizados no Art. 7 da Lei MP
dictionaryKeywordsSexualViolence = ["estuprar", "abusar", "assediar", "agarrar"]
dictionaryKeywordsPhysicalViolence = ["bater", "machucar", "espancar", "empurrar"]
dictionaryKeywordsPsychologicalViolence = ["ameaçar", "humilhar", "xingar", "ofender"]
dictionaryKeywordsMoralViolence = ["acusar", "chantagear", "ofender", "caluniar"]
dictionaryKeywordsPatrimonialViolence = ["quebrar", "destruir", "pegar", "roubar"]

Da mesma forma que foi feita a listagem de palavras, também foram listados **pesos** para cada uma dessas palavras baseado na sua frequência nos relatos. Os que estão escritos a seguir foram adicionados apenas para simulação e também deverão ser a entrada do sistema para que o usuário final possa adicioná-los:

In [ ]:
weightsForKeywordsSexualViolence = [4, 3, 2, 1]
weightsForKeywordsPhysicalViolence = [3, 2, 4, 1]
weightsForKeywordsPsychologicalViolence = [4, 2, 1, 3]
weightsForKeywordsMoralViolence = [2, 4, 1, 3]
weightsForKeywordsPatrimonialViolence = [2, 4, 1, 3]

A seguir estão listadas as **variáveis de autenticação** para acesso da *API do Twitter*, que são individuais e geradas automaticamente quando criado o projeto no *Dashboard* da sua conta de desenvolvedor da rede social:

In [ ]:
consumer_key = 'put your key here'
consumer_secret = 'put your key here'
token_key = 'put your key here'
token_secret = 'put your key here'

### Funções

A `accessApiTwitter` faz a autenticação utilizando as variáveis acima mostradas e retorna um objeto de **wrapper** para API:

In [ ]:
def accessApiTwitter():

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(token_key, token_secret)

    api = tweepy.API(auth,wait_on_rate_limit=True)

    return api

A `searchHashtagReportOnTwitter` recebe a `hashtagName` que será pesquisada, como exemplos

- #EuViviUmRelacionamentoAbusivo
- #maselenuncamebateu
- #MeuExAbusivo
- #érelacionamentoabusivoquando
- #elenãotebate

um `dateSince` que é a data de início para a pesquisa - que pegará todas as postagens feitas com `hashtagName` de `dateSince` até agora - e `accessPackageApiTwitter` que é o objeto de **wrapper** retornado na função anteior `accessApiTwitter`. A função vai preenchendo dentro do *loop* o **dataframe** `df` com cada postagem pegando sua data de criação e texto:   


In [ ]:
def searchHashtagReportOnTwitter(hashtagName, dateSince, accessPackageApiTwitter):
    global df

    for tweet in tweepy.Cursor(accessPackageApiTwitter.search,
                               q=hashtagName,
                               lang="pt",
                               since=dateSince,
                               tweet_mode="extended").items(10):
        df = df.append({"data": tweet.created_at, "post": tweet.full_text}, ignore_index=True)

A função `reportTokenization` faz a tokenização (já personalizada) do tweet e retorna um `doc`:

In [ ]:
def reportTokenization(tweet):
    doc = nlp(tweet)
    return doc

![Hooking an arbitrary tokenizer into the pipeline](https://spacy.io/pipeline-7a14d4edd18f3edfee8f34393bff2992.svg "SpaCy Pipeline")

Em `selectOnlyVerbsReport` é colocado em uma `list` apenas os verbos de cada texto da postagem. O SpaCy tem um recurso `pos_` do objeto `Token` que diz qual seu tipo, se é **VERB**, **NOUN** ou outras combinações como pode ser visto em [Linguistic Features - POS Tagging](https://spacy.io/usage/linguistic-features#pos-tagging).  

In [ ]:
def selectOnlyVerbsReport(doc):
    list = []
    for token in doc:
        if token.pos_ == 'VERB':
            list.append(token)

    return list

Alguns opções de visualizações:

![Visualizing the dependency parse](https://spacy.io/displacy-3504502e1d5463ede765f0a789717424.svg "Visualizing the dependency parse")

![Visualizing the dependency parse](https://spacy.io/displacy-compact-4c063e533e7ca1019a2f763ed5b7a925.svg "Visualizing the dependency parse")



O `sumOfList` serve apenas para fazer a soma dos valores em uma lista:

In [ ]:
def sumOfList(list):
    sum = 0
    for num in list:
        sum += num

    return sum

Em `createNumericRepresentation` o objetivo é retornar uma **representação numérica** do texto para cada tipo de violência. Ou seja, atribuir uma representação de similaridade para cada verbo no dicionário em comparação aos verbos do texto em análise. Ter essas representações irá nos ajudar a entender com qual tipo de violência o relato mais se assemelha. Observe que os tamanhos das `listNumericRepresentation` retornadas são sempre iguais ao tamanho do dicionário, porque é um tamanho que pode ser fixado e se tornar padrão já que a quantidade de verbos por relatos não têm um tamanho fixo. Ao final do cálculo da média de similaridade, é feita uma multiplicação com o peso de cada palavra do dicionário que também são a entrada no nosso sistema como já foi explicado: 

In [ ]:
def createNumericRepresentation(list, dictionary, weight):
    listNumericRepresentation = []
    nlp = spacy.load("pt_core_news_sm")

    for index, word in enumerate(dictionary):
        doc = nlp(word)
        sum = 0
        for token in list:
            sum += doc.similarity(token)
        mean = sum / len(list)

        listNumericRepresentation.append(mean*weight[index])

    return listNumericRepresentation

Quando retornado as listas, teremos para cada dicionário uma representação numérica do texto em cima desses dicionários, como exemplo:

- ["estuprar", "abusar", "assediar", "agarrar"] => [0.8, 0.5, 0.3, 0.1]
- ["bater", "machucar", "espancar", "empurrar"] => [0.7, 0.5, 0.2, 0.1]
- ["ameaçar", "humilhar", "xingar", "ofender"] => [0.3, 0.5, 0.9, 0.2]
- ["acusar", "chantagear", "ofender", "caluniar"] => [0.4, 0.6, 0.0, 0.9]
- ["quebrar", "destruir", "pegar", "roubar"] => [0.8, 0.7, 0.3, 0.2]

e na função `typeAnalysis` preenchemos uma *list* `listViolence` de objetos `Violence` registrados no arquivo `violence.py` explicado no primeiro item desse documento. Depois é feito um `sorted` com o `value` de cada `type` e rankeado os tipos do mais provável para o menos provável retornando em `typeViolenceSorted`. 

In [ ]:
def typeAnalysis(list):

    listViolence = []

    listNumRepresSexualViolence = createNumericRepresentation(list, dictionaryKeywordsSexualViolence, weightsForKeywordsSexualViolence)
    listViolence.append(Violence(-sumOfList(listNumRepresSexualViolence), 'Violência Sexual'))
    
    listNumRepresPhysicalViolence = createNumericRepresentation(list, dictionaryKeywordsPhysicalViolence, weightsForKeywordsPhysicalViolence)
    listViolence.append(Violence(-sumOfList(listNumRepresPhysicalViolence), 'Violência Física'))

    listNumRepresPsychologicalViolence = createNumericRepresentation(list, dictionaryKeywordsPsychologicalViolence, weightsForKeywordsPsychologicalViolence)
    listViolence.append(Violence(-sumOfList(listNumRepresPsychologicalViolence), 'Violência Psicológica'))

    listNumRepresMoralViolence = createNumericRepresentation(list, dictionaryKeywordsMoralViolence, weightsForKeywordsMoralViolence)
    listViolence.append(Violence(-sumOfList(listNumRepresMoralViolence), 'Violência Moral'))

    listNumRepresPatrimonialViolence = createNumericRepresentation(list, dictionaryKeywordsPatrimonialViolence, weightsForKeywordsPatrimonialViolence)
    listViolence.append(Violence(-sumOfList(listNumRepresPatrimonialViolence), 'Violência Patrimonial'))

    typeViolenceSorted = sorted(listViolence, key=Violence.get_value)

    return typeViolenceSorted

### Principal

Aqui é utilizada a classe do arquivo `myTweetTokenizer.py` - mostrado no segundo item desse documento - para retornar e inserir um tokenizer específico na estrutura padrão do SpaCy:

In [ ]:
nlp = spacy.load("pt_core_news_sm")
nlp.tokenizer = MyTweetTokenizer(nlp.vocab)

Segue a iteração por todas as linhas do *DataFrame* selecionando `row[1]` onde está o texto da postagem:

In [ ]:
for index, row in df.iterrows():
    doc = reportTokenization(row[1])

É selecionado apenas o `type` do retorno da função `typeAnalysis` e adicionado no *DataFrame* `df`:

In [ ]:
typeViolenceSorted = [v.get_type() for v in valueAndTypeViolenceSorted]
    df.at[index, 'tipoDeViolencia'] = typeViolenceSorted

E finalmente o *DataFrame* é registrado no `csv`:

In [ ]:
df.to_csv('datasetReports.csv')